## Neural Part Of Speech Tagging

We're now going to solve the same problem of POS tagging with neural networks.
<img src=https://i.stack.imgur.com/6pdIT.png width=320>

From deep learning perspective, this is a task of predicting a sequence of outputs aligned to a sequence of inputs. There are several problems that match this formulation:
* Part Of Speech Tagging -  an auxuliary task for many NLP problems
* Named Entity Recognition - for chat bots and web crawlers
* Protein structure prediction - for bioinformatics

In [1]:
%tensorflow_version 2.x
# import tensorflow

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import nltk
import sys
import numpy as np

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
<ipython-input-2-bc9847c14597>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])


In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [4]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))


draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


### Building vocabularies

Just like before, we have to build a mapping from tokens to integer ids. This time around, our model operates on a word level, processing one word per RNN step. This means we'll have to deal with far larger vocabulary.

Luckily for us, we only receive those words as input i.e. we don't have to predict them. This means we can have a large vocabulary for free by using word embeddings.

In [5]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

#let's measure what fraction of data words are in the dictionary
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [6]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

convert words and tags into fixed-size matrix

In [7]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""

    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix

In [8]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

### Build model

Unlike our previous lab, this time we'll focus on a high-level keras interface to recurrent neural networks. It is as simple as you can get with RNN, allbeit somewhat constraining for complex tasks like seq2seq.

By default, all keras RNNs apply to a whole sequence of inputs and produce a sequence of hidden states `(return_sequences=True` or just the last hidden state `(return_sequences=False)`. All the recurrence is happening under the hood.

At the top of our model we need to apply a Dense layer to each time-step independently. As of now, by default keras.layers.Dense would apply once to all time-steps concatenated. We use __keras.layers.TimeDistributed__ to modify Dense layer so that it would apply across both batch and time axes.

In [9]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

#add top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

__Training:__ in this case we don't want to prepare the whole training dataset in advance. The main cause is that the length of every batch depends on the maximum sentence length within the batch. This leaves us two options: use custom training code as in previous seminar or use generators.

Keras models have a __`model.fit_generator`__ method that accepts a python generator yielding one batch at a time. But first we need to implement such generator:

In [10]:
import tensorflow
from tensorflow.keras.utils import to_categorical

BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"

    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot


__Callbacks:__ Another thing we need is to measure model performance. The tricky part is not to count accuracy after sentence ends (on padding) and making sure we count all the validation data exactly once.

While it isn't impossible to persuade Keras to do all of that, we may as well write our own callback that does that.
Keras callbacks allow you to write a custom code to be ran once every epoch or every minibatch. We'll define one via LambdaCallback

In [11]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()

In [12]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


<ipython-input-12-e75a6e7fae6a>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1343/1343 [============================>.] - ETA: 0s - loss: 0.2514
Measuring validation accuracy...
448/448 [==============================] - 7s 14ms/step

Validation accuracy: 0.94038

1343/1343 [==============================] - 130s 92ms/step - loss: 0.2513
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0582
Measuring validation accuracy...
448/448 [==============================] - 6s 14ms/step

Validation accuracy: 0.94451

1343/1343 [==============================] - 91s 68ms/step - loss: 0.0582
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0513
Measuring validation accuracy...
448/448 [==============================] - 6s 14ms/step

Validation accuracy: 0.94564

1343/1343 [==============================] - 84s 62ms/step - loss: 0.0513
Epoch 4/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0472
Measuring validation accuracy...
448/448 [==============================] - 7s 16ms/step

Validation accuracy: 0.94

Measure final accuracy on the whole test set.

In [13]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

448/448 [==============================] - 6s 14ms/step
Final accuracy: 0.94586


### Going bidirectional

Since we're analyzing a full sequence, it's legal for us to look into future data.

A simple way to achieve that is to go both directions at once, making a __bidirectional RNN__.

In Keras you can achieve that both manually (using two LSTMs and Concatenate) and by using __`keras.layers.Bidirectional`__.

This one works just as `TimeDistributed` we saw before: you wrap it around a recurrent layer (SimpleRNN now and LSTM/GRU later) and it actually creates two layers under the hood.

Your first task is to use such a layer our POS-tagger.

In [42]:
# Функция определения модели с изменяемым обертывающим слоем

In [29]:
def models (recurrent_layer):
  model = keras.models.Sequential()
  model.add(L.InputLayer([None],dtype='int32'))
  model.add(L.Embedding(len(all_words),50))

  # в исходную модель добавили слой Bidirectional обертывающий слой SimpleRNN
  model.add(L.Bidirectional(recurrent_layer(64, return_sequences=True)))

  # Add the top layer that predicts tag probabilities
  stepwise_dense = L.Dense(len(all_tags),activation='softmax')
  stepwise_dense = L.TimeDistributed(stepwise_dense)
  model.add(stepwise_dense)
  return model


In [43]:
# 1. Bidirectional обертывающий слой SimpleRNN

In [30]:
model = models(L.SimpleRNN)

In [31]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


<ipython-input-31-e75a6e7fae6a>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.1882
Measuring validation accuracy...
448/448 [==============================] - 12s 27ms/step

Validation accuracy: 0.95649

1343/1343 [==============================] - 224s 165ms/step - loss: 0.1882
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0421
Measuring validation accuracy...
448/448 [==============================] - 12s 27ms/step

Validation accuracy: 0.96104

1343/1343 [==============================] - 170s 127ms/step - loss: 0.0421
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0350
Measuring validation accuracy...
448/448 [==============================] - 12s 27ms/step

Validation accuracy: 0.96249

1343/1343 [==============================] - 162s 121ms/step - loss: 0.0350
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0297
Measuring validation accuracy...
448/448 [==============================] - 12s 27ms/step

Validation accur

In [33]:
acc_bslrn = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc_bslrn)

assert acc_bslrn>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 12s 27ms/step

Final accuracy: 0.96232
Well done!


In [20]:
# 2. Bidirectional обертывающий слой LSTM

In [35]:
model_blstm = models(L.LSTM)

model_blstm.compile('adam','categorical_crossentropy')

model_blstm.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


<ipython-input-35-bbdb32a031f2>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_blstm.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2688
Measuring validation accuracy...
448/448 [==============================] - 4s 7ms/step

Validation accuracy: 0.95517

1343/1343 [==============================] - 50s 34ms/step - loss: 0.2688
Epoch 2/5
1340/1343 [============================>.] - ETA: 0s - loss: 0.0445
Measuring validation accuracy...
448/448 [==============================] - 2s 5ms/step

Validation accuracy: 0.96111

1343/1343 [==============================] - 19s 14ms/step - loss: 0.0445
Epoch 3/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0367
Measuring validation accuracy...
448/448 [==============================] - 3s 7ms/step

Validation accuracy: 0.96370

1343/1343 [==============================] - 19s 14ms/step - loss: 0.0367
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0320
Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation accuracy: 0.96422



In [36]:
acc_blstm = compute_test_accuracy(model_blstm)
print("\nFinal accuracy: %.5f"%acc_blstm)

assert acc_blstm>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 3s 6ms/step

Final accuracy: 0.96535
Well done!


In [37]:
# 3. Bidirectional обертывающий слой GRU

In [38]:
model_gru = models(L.GRU)

model_gru.compile('adam','categorical_crossentropy')

model_gru.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


<ipython-input-38-ec75fc261529>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_gru.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1341/1343 [============================>.] - ETA: 0s - loss: 0.1911
Measuring validation accuracy...
448/448 [==============================] - 3s 5ms/step

Validation accuracy: 0.95637

1343/1343 [==============================] - 61s 42ms/step - loss: 0.1908
Epoch 2/5
1340/1343 [============================>.] - ETA: 0s - loss: 0.0423
Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation accuracy: 0.96266

1343/1343 [==============================] - 20s 15ms/step - loss: 0.0423
Epoch 3/5
1341/1343 [============================>.] - ETA: 0s - loss: 0.0353
Measuring validation accuracy...
448/448 [==============================] - 2s 5ms/step

Validation accuracy: 0.96458

1343/1343 [==============================] - 16s 12ms/step - loss: 0.0353
Epoch 4/5
1338/1343 [============================>.] - ETA: 0s - loss: 0.0310
Measuring validation accuracy...
448/448 [==============================] - 2s 5ms/step

Validation accuracy: 0.96575



In [39]:
acc_gru = compute_test_accuracy(model_gru)
print("\nFinal accuracy: %.5f"%acc_gru)

assert acc_gru>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 2s 5ms/step

Final accuracy: 0.96590
Well done!


In [40]:
# Сравнение Final accuracy

In [41]:
from prettytable import PrettyTable

In [49]:
table = PrettyTable()
table.field_names = ["Слой Bidirectional обертывающий слой:", "Final accuracy"]
table.add_row(["SimpleRNN", "\nFinal accuracy: %.5f"%acc_bslrn])
table.add_row(["LSTM", "\nFinal accuracy: %.5f"%acc_blstm])
table.add_row(["GRU", "\nFinal accuracy: %.5f"%acc_gru])
table.valign = "m"
print(table.get_string())

+---------------------------------------+-------------------------+
| Слой Bidirectional обертывающий слой: |      Final accuracy     |
+---------------------------------------+-------------------------+
|               SimpleRNN               |                         |
|                                       | Final accuracy: 0.96232 |
|                  LSTM                 |                         |
|                                       | Final accuracy: 0.96535 |
|                  GRU                  |                         |
|                                       | Final accuracy: 0.96590 |
+---------------------------------------+-------------------------+


Create **at least one experiment** from the list bellow, you can choose the most interesting and promising options to improve the performance of Bidirectional LSTM:

* __Go beyond SimpleRNN__: there's `keras.layers.LSTM` and `keras.layers.GRU`
  * If you want to use a custom recurrent Cell, read [this](https://keras.io/layers/recurrent/#rnn)
  * You can also use 1D Convolutions (`keras.layers.Conv1D`). They are often as good as recurrent layers but with less overfitting.
* __Stack more layers__: if there is a common motif to this course it's about stacking layers
  * You can just add recurrent and 1dconv layers on top of one another and keras will understand it
  * Just remember that bigger networks may need more epochs to train
* __Regularization__: you can apply dropouts as usual but also in an RNN-specific way
  * `keras.layers.Dropout` works inbetween RNN layers
  * Recurrent layers also have `recurrent_dropout` parameter
* __Gradient clipping__: If your training isn't as stable as you'd like, set `clipnorm` in your optimizer.
  * Which is to say, it's a good idea to watch over your loss curve at each minibatch. Try tensorboard callback or something similar.

In [87]:
# Так как в прошлых эксперементах, GRU показал лучший результат буду использовать его для улучшений, а так же поэкспериметирую с сочитанием GRU и SLTM

In [84]:
model_final = keras.models.Sequential()
model_final.add(L.InputLayer([None],dtype='int32'))
model_final.add(L.Embedding(len(all_words),50))

# model_final.add(L.Bidirectional(L.GRU(64, dropout=0.2, return_sequences=True)))
model_final.add(L.Bidirectional(L.LSTM(64, dropout=0.2, return_sequences=True)))

# model_final.add(L.Dropout(0.5))
# добавим еще один слой Bidirectional GRU
model_final.add(L.Bidirectional(L.GRU(64, dropout=0.2, return_sequences=True)))

# Add the top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model_final.add(stepwise_dense)

In [85]:
model_final.compile('adam','categorical_crossentropy')

model_final.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=10,)

Epoch 1/10


<ipython-input-85-a8ec3bed2e85>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2228
Measuring validation accuracy...
448/448 [==============================] - 6s 11ms/step

Validation accuracy: 0.95547

1343/1343 [==============================] - 61s 40ms/step - loss: 0.2228
Epoch 2/10
1343/1343 [============================>.] - ETA: 0s - loss: 0.0499
Measuring validation accuracy...
448/448 [==============================] - 4s 9ms/step

Validation accuracy: 0.96004

1343/1343 [==============================] - 28s 21ms/step - loss: 0.0499
Epoch 3/10
1342/1343 [============================>.] - ETA: 0s - loss: 0.0418
Measuring validation accuracy...
448/448 [==============================] - 4s 9ms/step

Validation accuracy: 0.96311

1343/1343 [==============================] - 26s 19ms/step - loss: 0.0417
Epoch 4/10
1342/1343 [============================>.] - ETA: 0s - loss: 0.0372
Measuring validation accuracy...
448/448 [==============================] - 4s 9ms/step

Validation accuracy: 0.964

In [86]:
acc_final = compute_test_accuracy(model_final)
print("\nFinal accuracy: %.5f"%acc_final)

assert acc_final>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 4s 10ms/step

Final accuracy: 0.96678
Well done!


In [90]:
# После проведенных экспериментов получилась следующая картина, при использовании двух рек.слоев GRU макс. accuracy = 0.96644
# При двух рек.слоях выросло на 0,3% это не много но прогресс есть :)